In [ ]:
# Imports 
import sys
sys.path.append("..")
import os
os.environ["pyna_language"] = 'python'
from julia.OpenMDAO import make_component
import julia.Main as julia
julia.include("source_comp.jl")
import pdb
import numpy as np
import pandas as pd
import openmdao.api as om
import matplotlib.pyplot as plt
%matplotlib inline
from pyNA.pyna import pyna

from IPython.display import clear_output
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)


Unable to import petsc4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.


In [ ]:
# Create pyna_settings 
case = 'stca'

# Directories and file names
pyna_settings = pyna.load_settings('stca')
pyna_settings.case_name = 'stca'                             # Case name [-]
pyna_settings.engine_file_name = 'engine_stca.csv'           # File name of the take-off engine inputs [-] 

py = pyna(pyna_settings)

In [ ]:
stca = py.load_results('../cases/NASA STCA Standard/output/Trajectory_stca.sql', 'final')
n_pp = np.where(stca.get_val('noise.propagation.x') > 6500)[0][0]

### Source module

In [ ]:
def get_indices_noise_input_vector(settings, n_t):
    """
    Get (julia) indices for input vector noise model.
    """

    # Initialize indices dictionary
    idx = dict()
    idx_src = dict()

    if settings.language == 'julia':
        idx["x"] = [0 * n_t + 1, 1 * n_t]
        idx["y"] = [1 * n_t + 1, 2 * n_t]
        idx["z"] = [2 * n_t + 1, 3 * n_t]
        idx["alpha"] = [3 * n_t + 1, 4 * n_t]
        idx["gamma"] = [4 * n_t + 1, 5 * n_t]
        idx["t_s"] = [5 * n_t + 1, 6 * n_t]
        idx["rho_0"] = [6 * n_t + 1, 7 * n_t]
        idx["mu_0"] = [7 * n_t + 1, 8 * n_t]
        idx["c_0"] = [8 * n_t + 1, 9 * n_t]
        idx["T_0"] = [9 * n_t + 1, 10 * n_t]
        idx["p_0"] = [10 * n_t + 1, 11 * n_t]
        idx["M_0"] = [11 * n_t + 1, 12 * n_t]
        idx["I_0"] = [12 * n_t + 1, 13 * n_t]
        idx["TS"] = [13 * n_t + 1, 14 * n_t]

        idx_src["TS"] = [0 * n_t + 1, 1 * n_t]
        idx_src["M_0"] = [1 * n_t + 1, 2 * n_t]
        idx_src["c_0"] = [2 * n_t + 1, 3 * n_t]
        idx_src["rho_0"] = [3 * n_t + 1, 4 * n_t]
        idx_src["mu_0"] = [4 * n_t + 1, 5 * n_t]
        idx_src["T_0"] = [5 * n_t + 1, 6 * n_t]
        idx_src["theta"] = [6 * n_t + 1, 7 * n_t]
        idx_src["phi"] = [7 * n_t + 1, 8 * n_t]

        n = 14
        n_src = 8
        if settings.jet_mixing == True and settings.jet_shock == False:
            idx["V_j"]   = [n * n_t + 1, (n + 1) * n_t]
            idx["rho_j"] = [(n + 1) * n_t + 1, (n + 2) * n_t]
            idx["A_j"]   = [(n + 2) * n_t + 1, (n + 3) * n_t]
            idx["Tt_j"]  = [(n + 3) * n_t + 1, (n + 4) * n_t]
            n = n + 4
            idx_src["V_j_star"]   = [n_src * n_t + 1, (n_src + 1) * n_t]
            idx_src["rho_j_star"] = [(n_src + 1) * n_t + 1, (n_src + 2) * n_t]
            idx_src["A_j_star"]   = [(n_src + 2) * n_t + 1, (n_src + 3) * n_t]
            idx_src["Tt_j_star"]  = [(n_src + 3) * n_t + 1, (n_src + 4) * n_t]
            n_src = n_src + 4
        elif settings.jet_shock == True and settings.jet_mixing == False:
            idx["V_j"]  = [n * n_t + 1, (n + 1) * n_t]
            idx["M_j"]       = [(n + 1) * n_t + 1, (n + 2) * n_t]
            idx["A_j"]  = [(n + 2) * n_t + 1, (n + 3) * n_t]
            idx["Tt_j"] = [(n + 3) * n_t + 1, (n + 4) * n_t]
            n = n + 4
            idx_src["V_j_star"] = [n_src * n_t + 1, (n_src + 1) * n_t]
            idx_src["M_j"] = [(n_src + 1) * n_t + 1, (n_src + 2) * n_t]
            idx_src["A_j_star"] = [(n_src + 2) * n_t + 1, (n_src + 3) * n_t]
            idx_src["Tt_j_star"] = [(n_src + 3) * n_t + 1, (n_src + 4) * n_t]
            n_src = n_src + 4
        elif settings.jet_shock == True and settings.jet_mixing == True:
            idx["V_j"] = [n * n_t + 1, (n + 1) * n_t]
            idx["rho_j"] = [(n + 1) * n_t + 1, (n + 2) * n_t]
            idx["A_j"] = [(n + 2) * n_t + 1, (n + 3) * n_t]
            idx["Tt_j"] = [(n + 3) * n_t + 1, (n + 4) * n_t]
            idx["M_j"] = [(n + 4) * n_t + 1, (n + 5) * n_t]
            n = n + 5
            idx_src["V_j_star"] = [n_src * n_t + 1, (n_src + 1) * n_t]
            idx_src["rho_j_star"] = [(n_src + 1) * n_t + 1, (n_src + 2) * n_t]
            idx_src["A_j_star"] = [(n_src + 2) * n_t + 1, (n_src + 3) * n_t]
            idx_src["Tt_j_star"] = [(n_src + 3) * n_t + 1, (n_src + 4) * n_t]
            idx_src["M_j"] = [(n_src + 4) * n_t + 1, (n_src + 5) * n_t]
            n_src = n_src + 5
        if settings.core:
            if settings.method_core_turb == "GE":
                idx["mdoti_c"] = [n * n_t + 1, (n + 1) * n_t]
                idx["Tti_c"] = [(n + 1) * n_t + 1, (n + 2) * n_t]
                idx["Ttj_c"] = [(n + 2) * n_t + 1, (n + 3) * n_t]
                idx["Pti_c"] = [(n + 3) * n_t + 1, (n + 4) * n_t]
                idx["DTt_des_c"] = [(n + 4) * n_t + 1, (n + 5) * n_t]
                n = n + 5
                idx_src["mdoti_c_star"] = [n_src * n_t + 1, (n_src + 1) * n_t]
                idx_src["Tti_c_star"] = [(n_src + 1) * n_t + 1, (n_src + 2) * n_t]
                idx_src["Ttj_c_star"] = [(n_src + 2) * n_t + 1, (n_src + 3) * n_t]
                idx_src["Pti_c_star"] = [(n_src + 3) * n_t + 1, (n_src + 4) * n_t]
                idx_src["DTt_des_c_star"] = [(n_src + 4) * n_t + 1, (n_src + 5) * n_t]
                n_src = n_src + 5
            elif settings.method_core_turb == "PW":
                idx["mdoti_c"] = [n * n_t + 1, (n + 1) * n_t]
                idx["Tti_c"] = [(n + 1) * n_t + 1, (n + 2) * n_t]
                idx["Ttj_c"] = [(n + 2) * n_t + 1, (n + 3) * n_t]
                idx["Pti_c"] = [(n + 3) * n_t + 1, (n + 4) * n_t]
                idx["rho_te_c"] = [(n + 4) * n_t + 1, (n + 5) * n_t]
                idx["c_te_c"] = [(n + 5) * n_t + 1, (n + 6) * n_t]
                idx["rho_ti_c"] = [(n + 6) * n_t + 1, (n + 7) * n_t]
                idx["c_ti_c"] = [(n + 7) * n_t + 1, (n + 8) * n_t]
                n = n + 8
                idx_src["mdoti_c_star"] = [n_src * n_t + 1, (n_src + 1) * n_t]
                idx_src["Tti_c_star"] = [(n_src + 1) * n_t + 1, (n_src + 2) * n_t]
                idx_src["Ttj_c_star"] = [(n_src + 2) * n_t + 1, (n_src + 3) * n_t]
                idx_src["Pti_c_star"] = [(n_src + 3) * n_t + 1, (n_src + 4) * n_t]
                idx_src["rho_te_c_star"] = [(n_src + 4) * n_t + 1, (n_src + 5) * n_t]
                idx_src["c_te_c_star"] = [(n_src + 5) * n_t + 1, (n_src + 6) * n_t]
                idx_src["rho_ti_c_star"] = [(n_src + 6) * n_t + 1, (n_src + 7) * n_t]
                idx_src["c_ti_c_star"] = [(n_src + 7) * n_t + 1, (n_src + 8) * n_t]
                n_src = n_src + 8
        if settings.airframe:
            idx["theta_flaps"] = [n * n_t + 1, (n + 1) * n_t]
            idx["I_landing_gear"] = [(n + 1) * n_t + 1, (n + 2) * n_t]
            n = n + 2
            idx_src["theta_flaps"] = [n_src * n_t + 1, (n_src + 1) * n_t]
            idx_src["I_landing_gear"] = [(n_src + 1) * n_t + 1, (n_src + 2) * n_t]
            n_src = n_src + 2
        if settings.fan_inlet == True or settings.fan_discharge == True:
            idx["DTt_f"] = [n * n_t + 1, (n + 1) * n_t]
            idx["mdot_f"] = [(n + 1) * n_t + 1, (n + 2) * n_t]
            idx["N_f"] = [(n + 2) * n_t + 1, (n + 3) * n_t]
            idx["A_f"] = [(n + 3) * n_t + 1, (n + 4) * n_t]
            idx["d_f"] = [(n + 4) * n_t + 1, (n + 5) * n_t]
            idx_src["DTt_f_star"] = [n_src * n_t + 1, (n_src + 1) * n_t]
            idx_src["mdot_f_star"] = [(n_src + 1) * n_t + 1, (n_src + 2) * n_t]
            idx_src["N_f_star"] = [(n_src + 2) * n_t + 1, (n_src + 3) * n_t]
            idx_src["A_f_star"] = [(n_src + 3) * n_t + 1, (n_src + 4) * n_t]
            idx_src["d_f_star"] = [(n_src + 4) * n_t + 1, (n_src + 5) * n_t]
    elif settings.language == 'python':
        idx["x"] = [0 * n_t, 1 * n_t]
        idx["y"] = [1 * n_t, 2 * n_t]
        idx["z"] = [2 * n_t, 3 * n_t]
        idx["alpha"] = [3 * n_t, 4 * n_t]
        idx["gamma"] = [4 * n_t, 5 * n_t]
        idx["t_s"] = [5 * n_t, 6 * n_t]
        idx["rho_0"] = [6 * n_t, 7 * n_t]
        idx["mu_0"] = [7 * n_t, 8 * n_t]
        idx["c_0"] = [8 * n_t, 9 * n_t]
        idx["T_0"] = [9 * n_t, 10 * n_t]
        idx["p_0"] = [10 * n_t, 11 * n_t]
        idx["M_0"] = [11 * n_t, 12 * n_t]
        idx["I_0"] = [12 * n_t, 13 * n_t]
        idx["TS"] = [13 * n_t, 14 * n_t]

        idx_src["TS"] = [0 * n_t, 1 * n_t]
        idx_src["M_0"] = [1 * n_t, 2 * n_t]
        idx_src["c_0"] = [2 * n_t, 3 * n_t]
        idx_src["rho_0"] = [3 * n_t, 4 * n_t]
        idx_src["mu_0"] = [4 * n_t, 5 * n_t]
        idx_src["T_0"] = [5 * n_t, 6 * n_t]
        idx_src["theta"] = [6 * n_t, 7 * n_t]
        idx_src["phi"] = [7 * n_t, 8 * n_t]

        n = 14
        n_src = 8
        if settings.jet_mixing == True and settings.jet_shock == False:
            idx["V_j"]   = [n * n_t, (n + 1) * n_t]
            idx["rho_j"] = [(n + 1) * n_t, (n + 2) * n_t]
            idx["A_j"]   = [(n + 2) * n_t, (n + 3) * n_t]
            idx["Tt_j"]  = [(n + 3) * n_t, (n + 4) * n_t]
            n = n + 4
            idx_src["V_j_star"]   = [n_src * n_t, (n_src + 1) * n_t]
            idx_src["rho_j_star"] = [(n_src + 1) * n_t, (n_src + 2) * n_t]
            idx_src["A_j_star"]   = [(n_src + 2) * n_t, (n_src + 3) * n_t]
            idx_src["Tt_j_star"]  = [(n_src + 3) * n_t, (n_src + 4) * n_t]
            n_src = n_src + 4
        elif settings.jet_shock == True and settings.jet_mixing == False:
            idx["V_j"]  = [n * n_t, (n + 1) * n_t]
            idx["M_j"]       = [(n + 1) * n_t, (n + 2) * n_t]
            idx["A_j"]  = [(n + 2) * n_t, (n + 3) * n_t]
            idx["Tt_j"] = [(n + 3) * n_t, (n + 4) * n_t]
            n = n + 4
            idx_src["V_j_star"] = [n_src * n_t, (n_src + 1) * n_t]
            idx_src["M_j"] = [(n_src + 1) * n_t, (n_src + 2) * n_t]
            idx_src["A_j_star"] = [(n_src + 2) * n_t, (n_src + 3) * n_t]
            idx_src["Tt_j_star"] = [(n_src + 3) * n_t, (n_src + 4) * n_t]
            n_src = n_src + 4
        elif settings.jet_shock == True and settings.jet_mixing == True:
            idx["V_j"] = [n * n_t, (n + 1) * n_t]
            idx["rho_j"] = [(n + 1) * n_t, (n + 2) * n_t]
            idx["A_j"] = [(n + 2) * n_t, (n + 3) * n_t]
            idx["Tt_j"] = [(n + 3) * n_t, (n + 4) * n_t]
            idx["M_j"] = [(n + 4) * n_t, (n + 5) * n_t]
            n = n + 5
            idx_src["V_j_star"] = [n_src * n_t, (n_src + 1) * n_t]
            idx_src["rho_j_star"] = [(n_src + 1) * n_t, (n_src + 2) * n_t]
            idx_src["A_j_star"] = [(n_src + 2) * n_t, (n_src + 3) * n_t]
            idx_src["Tt_j_star"] = [(n_src + 3) * n_t, (n_src + 4) * n_t]
            idx_src["M_j"] = [(n_src + 4) * n_t, (n_src + 5) * n_t]
            n_src = n_src + 5
        if settings.core:
            if settings.method_core_turb == "GE":
                idx["mdoti_c"] = [n * n_t, (n + 1) * n_t]
                idx["Tti_c"] = [(n + 1) * n_t, (n + 2) * n_t]
                idx["Ttj_c"] = [(n + 2) * n_t, (n + 3) * n_t]
                idx["Pti_c"] = [(n + 3) * n_t, (n + 4) * n_t]
                idx["DTt_des_c"] = [(n + 4) * n_t, (n + 5) * n_t]
                n = n + 5
                idx_src["mdoti_c_star"] = [n_src * n_t, (n_src + 1) * n_t]
                idx_src["Tti_c_star"] = [(n_src + 1) * n_t, (n_src + 2) * n_t]
                idx_src["Ttj_c_star"] = [(n_src + 2) * n_t, (n_src + 3) * n_t]
                idx_src["Pti_c_star"] = [(n_src + 3) * n_t, (n_src + 4) * n_t]
                idx_src["DTt_des_c_star"] = [(n_src + 4) * n_t, (n_src + 5) * n_t]
                n_src = n_src + 5
            elif settings.method_core_turb == "PW":
                idx["mdoti_c"] = [n * n_t, (n + 1) * n_t]
                idx["Tti_c"] = [(n + 1) * n_t, (n + 2) * n_t]
                idx["Ttj_c"] = [(n + 2) * n_t, (n + 3) * n_t]
                idx["Pti_c"] = [(n + 3) * n_t, (n + 4) * n_t]
                idx["rho_te_c"] = [(n + 4) * n_t, (n + 5) * n_t]
                idx["c_te_c"] = [(n + 5) * n_t, (n + 6) * n_t]
                idx["rho_ti_c"] = [(n + 6) * n_t, (n + 7) * n_t]
                idx["c_ti_c"] = [(n + 7) * n_t, (n + 8) * n_t]
                n = n + 8
                idx_src["mdoti_c_star"] = [n_src * n_t, (n_src + 1) * n_t]
                idx_src["Tti_c_star"] = [(n_src + 1) * n_t, (n_src + 2) * n_t]
                idx_src["Ttj_c_star"] = [(n_src + 2) * n_t, (n_src + 3) * n_t]
                idx_src["Pti_c_star"] = [(n_src + 3) * n_t, (n_src + 4) * n_t]
                idx_src["rho_te_c_star"] = [(n_src + 4) * n_t, (n_src + 5) * n_t]
                idx_src["c_te_c_star"] = [(n_src + 5) * n_t, (n_src + 6) * n_t]
                idx_src["rho_ti_c_star"] = [(n_src + 6) * n_t, (n_src + 7) * n_t]
                idx_src["c_ti_c_star"] = [(n_src + 7) * n_t, (n_src + 8) * n_t]
                n_src = n_src + 8
        if settings.airframe:
            idx["theta_flaps"] = [n * n_t, (n + 1) * n_t]
            idx["I_landing_gear"] = [(n + 1) * n_t, (n + 2) * n_t]
            n = n + 2
            idx_src["theta_flaps"] = [n_src * n_t, (n_src + 1) * n_t]
            idx_src["I_landing_gear"] = [(n_src + 1) * n_t, (n_src + 2) * n_t]
            n_src = n_src + 2
        if settings.fan_inlet == True or settings.fan_discharge == True:
            idx["DTt_f"] = [n * n_t, (n + 1) * n_t]
            idx["mdot_f"] = [(n + 1) * n_t, (n + 2) * n_t]
            idx["N_f"] = [(n + 2) * n_t, (n + 3) * n_t]
            idx["A_f"] = [(n + 3) * n_t, (n + 4) * n_t]
            idx["d_f"] = [(n + 4) * n_t, (n + 5) * n_t]
            idx_src["DTt_f_star"] = [n_src * n_t, (n_src + 1) * n_t]
            idx_src["mdot_f_star"] = [(n_src + 1) * n_t, (n_src + 2) * n_t]
            idx_src["N_f_star"] = [(n_src + 2) * n_t, (n_src + 3) * n_t]
            idx_src["A_f_star"] = [(n_src + 3) * n_t, (n_src + 4) * n_t]
            idx_src["d_f_star"] = [(n_src + 4) * n_t, (n_src + 5) * n_t]

    return idx, idx_src

In [ ]:
# Source module 
n_t = 1
py.settings.language = 'julia'
py.settings.jet_shock = False
py.settings.all_sources = False
py.settings.observer_lst = ('flyover')
py.settings.x_observer_array = np.array([[6500., 0., 4*0.3048]])
_, idx_src = get_indices_noise_input_vector(py.settings, n_t)
shielding = np.ones((210, 24))
prob_s = om.Problem()
comp = make_component(julia.Source(py.settings, py.noise.data, py.ac, shielding, n_t, idx_src))
prob_s.model.add_subsystem("s", comp)
prob_s.setup()

if py.settings.core:
    prob_s.set_val('s.mdoti_c_star', stca.get_val('noise.source.mdoti_c_star')[n_pp])
    prob_s.set_val('s.Tti_c_star', stca.get_val('noise.source.Tti_c_star')[n_pp])
    prob_s.set_val('s.Ttj_c_star', stca.get_val('noise.source.Ttj_c_star')[n_pp])
    prob_s.set_val('s.Pti_c_star', stca.get_val('noise.source.Pti_c_star')[n_pp])
    prob_s.set_val('s.DTt_des_c_star', stca.get_val('noise.source.DTt_des_c_star')[n_pp])
if py.settings.airframe:
    prob_s.set_val('s.theta_flaps', stca.get_val('noise.source.theta_flaps')[n_pp])
    prob_s.set_val('s.I_landing_gear', stca.get_val('noise.source.I_landing_gear')[n_pp])
if py.settings.jet_mixing or py.settings.jet_shock:
    prob_s.set_val('s.V_j_star', stca.get_val('noise.source.V_j_star')[n_pp])
    prob_s.set_val('s.rho_j_star', stca.get_val('noise.source.rho_j_star')[n_pp])
    prob_s.set_val('s.A_j_star', stca.get_val('noise.source.A_j_star')[n_pp])
    prob_s.set_val('s.Tt_j_star', stca.get_val('noise.source.Tt_j_star')[n_pp])
if py.settings.jet_shock:
    prob_s.set_val('s.M_j', stca.get_val('noise.source.M_j')[n_pp])
if py.settings.fan_inlet or py.settings.fan_discharge:
    prob_s.set_val('s.DTt_f_star', stca.get_val('noise.source.DTt_f_star')[n_pp])
    prob_s.set_val('s.mdot_f_star', stca.get_val('noise.source.mdot_f_star')[n_pp])
    prob_s.set_val('s.N_f_star', stca.get_val('noise.source.N_f_star')[n_pp])
    prob_s.set_val('s.A_f_star', stca.get_val('noise.source.A_f_star')[n_pp])
    prob_s.set_val('s.d_f_star', stca.get_val('noise.source.d_f_star')[n_pp])
prob_s.set_val('s.TS', stca.get_val('noise.source.TS')[n_pp])
prob_s.set_val('s.M_0', stca.get_val('noise.source.M_0')[n_pp])
prob_s.set_val('s.c_0', stca.get_val('noise.source.c_0')[n_pp])
prob_s.set_val('s.rho_0', stca.get_val('noise.source.rho_0')[n_pp])
prob_s.set_val('s.mu_0', stca.get_val('noise.source.mu_0')[n_pp])
prob_s.set_val('s.T_0', stca.get_val('noise.source.T_0')[n_pp])
prob_s.set_val('s.theta', stca.get_val('noise.source.theta')[0,n_pp])
prob_s.set_val('s.phi', stca.get_val('noise.source.phi')[0,n_pp])

# Run the source module
prob_s.run_model()
prob_s.check_partials(compact_print=True, method='fd')
msap_source = prob_s.get_val('s.msap_source')

In [ ]:
# Check partials 
var_lst = ['s.V_j_star', 's.rho_j_star', 's.A_j_star', 's.Tt_j_star', 
           's.DTt_f_star', 's.mdot_f_star', 's.N_f_star', 's.A_f_star',
           's.mdoti_c_star', 's.Tti_c_star', 's.Ttj_c_star', 's.Pti_c_star', 's.DTt_des_c_star']
dd = prob_s.compute_totals(of='s.msap_source', wrt=var_lst, return_format='array')

In [ ]:
# Figure 
fig, ax = plt.subplots(1,3,figsize=(20,4), dpi=300)
colors = plt.cm.magma(np.linspace(0,0.8,5))

plt.style.use('plot.mplstyle')
for i,var in enumerate(var_lst):
    if i in np.arange(4):
        if var == 's.V_j_star':
            ax[2].semilogx(py.noise.data.f, dd[:,i]*stca.get_val('noise.source.V_j_star')[n_pp]/msap_source.flatten(), '-', label='$V_j^*$', color=colors[0])
        if var == 's.rho_j_star':
            ax[2].semilogx(py.noise.data.f, dd[:,i]*stca.get_val('noise.source.rho_j_star')[n_pp]/msap_source.flatten(), '--', label=r'$\rho_j^*$', color=colors[1])
        if var == 's.A_j_star':
            ax[2].semilogx(py.noise.data.f, dd[:,i]*stca.get_val('noise.source.A_j_star')[n_pp]/msap_source.flatten(), '-.', label='$A_j^*$', color=colors[2])
        if var == 's.Tt_j_star':
            ax[2].semilogx(py.noise.data.f, dd[:,i]*stca.get_val('noise.source.Tt_j_star')[n_pp]/msap_source.flatten(), ':', label='$Tt_j^*$', color=colors[3])

    elif i in np.arange(4,8):
        if var == 's.DTt_f_star':
            ax[0].semilogx(py.noise.data.f, dd[:,i]*stca.get_val('noise.source.DTt_f_star')[n_pp]/msap_source.flatten(), '-', label='$\Delta Tt_{f}^*$', color=colors[0])
        elif var == 's.mdot_f_star':
            ax[0].semilogx(py.noise.data.f, dd[:,i]*stca.get_val('noise.source.mdot_f_star')[n_pp]/msap_source.flatten(), '--', label='$\dot{m}_f^*$', color=colors[1])
        elif var == 's.N_f_star':
            ax[0].semilogx(py.noise.data.f, dd[:,i]*stca.get_val('noise.source.N_f_star')[n_pp]/msap_source.flatten(), '-.', label='$N_f^*$', color=colors[2])
        elif var == 's.A_f_star':
            ax[0].semilogx(py.noise.data.f, dd[:,i]*stca.get_val('noise.source.A_f_star')[n_pp]/msap_source.flatten(), '-D', label='$A_f^*$', color=colors[4])
    elif i in np.arange(8, 13):
        if var == 's.mdoti_c_star':
            ax[1].semilogx(py.noise.data.f, dd[:,i]*stca.get_val('noise.source.mdoti_c_star')[n_pp]/msap_source.flatten(), '-', label='$\dot{m}_{c,i}^*$', color=colors[0])
        elif var == 's.Tti_c_star':
            ax[1].semilogx(py.noise.data.f, dd[:,i]*stca.get_val('noise.source.Tti_c_star')[n_pp]/msap_source.flatten(), '--', label='$Tt_{c,i}^*$', color=colors[1])
        elif var == 's.Ttj_c_star':
            ax[1].semilogx(py.noise.data.f, dd[:,i]*stca.get_val('noise.source.Ttj_c_star')[n_pp]/msap_source.flatten(), '-.', label='$Tt_{c,j}^*$', color=colors[2])
        elif var == 's.Pti_c_star':
            ax[1].semilogx(py.noise.data.f, dd[:,i]*stca.get_val('noise.source.Pti_c_star')[n_pp]/msap_source.flatten(), ':', label='$Pt_{c,i}^*$', color=colors[3])
        elif var == 's.DTt_des_c_star':
            ax[1].semilogx(py.noise.data.f, dd[:,i]*stca.get_val('noise.source.DTt_des_c_star')[n_pp]/msap_source.flatten(), '-D', label='$\Delta Tt_{des,c}^*$', color=colors[4])

ax[0].set_ylim([-2.25, 2.25])
ax[1].set_ylim([-2.25, 2.25])

ax[0].set_title('Fan')
ax[0].set_xlabel('Frequency [Hz]')
ax[0].set_ylabel(r'$\frac{\partial \langle p^2 \rangle_{source}}{\partial Y_i} \ \frac{[\%]}{[\%]}$', fontsize=18)
ax[1].set_title('Core')
ax[1].set_xlabel('Frequency [Hz]')
ax[1].set_ylabel(r'$\frac{\partial \langle p^2 \rangle_{source}}{\partial Y_i} \ \frac{[\%]}{[\%]}$', fontsize=18)
ax[2].set_title('Jet mixing')
ax[2].set_xlabel('Frequency [Hz]')
ax[2].set_ylabel(r'$\frac{\partial \langle p^2 \rangle_{source}}{\partial Y_i} \ \frac{[\%]}{[\%]}$', fontsize=18)

ax[0].plot([7707.57, 7707.57], [-2.25, 2.25], 'k-', linewidth=1.5)
ax[0].annotate(xy=(6000, -0.55), s='BPF', fontsize=14, backgroundcolor='w')

ax[1].plot([400, 400], [-2.25, 2.25], 'k-', linewidth=1.5)
ax[1].annotate(xy=(350, -1.5), s='$f_p$', fontsize=14, backgroundcolor='w')

ax[0].legend(fontsize=12, loc='lower left', bbox_to_anchor=(0.03, 0.765), ncol=2, borderaxespad=0, frameon=False)
ax[1].legend(fontsize=12, loc='lower left', bbox_to_anchor=(0.45, 0.65), ncol=2, borderaxespad=0, frameon=False)
ax[2].legend(fontsize=12, loc='lower left', bbox_to_anchor=(0.80, 0.55), ncol=1, borderaxespad=0, frameon=False)
plt.subplots_adjust(wspace=0.25)

### Propagation module

In [ ]:
# Create problem
py.settings.n_t = 1
prob_p = om.Problem()
comp = make_component(julia.Propagation(py.settings, py.noise.data, x_obs))
prob_p.model.add_subsystem('p', comp)
prob_p.setup(force_alloc_complex=True)

prob_p.set_val('p.r', stca.get_val('flyover.propagation.r')[n_pp])
prob_p.set_val('p.z', stca.get_val('flyover.propagation.z')[n_pp])
prob_p.set_val('p.c_bar', stca.get_val('flyover.propagation.c_bar')[n_pp])
prob_p.set_val('p.rho_0', stca.get_val('flyover.propagation.rho_0')[n_pp])
prob_p.set_val('p.I_0', stca.get_val('flyover.propagation.I_0')[n_pp])
prob_p.set_val('p.beta', stca.get_val('flyover.propagation.beta')[n_pp])

f_24 = np.array([   50.11872336,    63.09573445,    79.43282347,   100.        ,
         125.89254118,   158.48931925,   199.5262315 ,   251.18864315,
         316.22776602,   398.10717055,   501.18723363,   630.95734448,
         794.32823472,  1000.        ,  1258.92541179,  1584.89319246,
        1995.26231497,  2511.88643151,  3162.27766017,  3981.07170553,
        5011.87233627,  6309.5734448 ,  7943.28234724, 10000.        ])
msap_source_i = np.zeros((np.size(stca.get_val('flyover.propagation.r')), pyna_settings.N_f))
for i in np.arange(np.size(stca.get_val('flyover.propagation.r'))):
    msap_source_i[i,:] = np.interp(py.noise.data.f, f_24, stca.get_val('flyover.propagation.msap_source')[i,:])

prob_p.set_val('p.msap_source', msap_source_i[n_pp, :])

# Run the propagation module
prob_p.run_model()
msap_prop = prob_p.get_val('p.msap_prop')

In [ ]:
# Check partials 
dd_p = prob_p.compute_totals(of='p.msap_prop', wrt='p.msap_source', return_format='array')

In [ ]:
fig = plt.figure(figsize=(15,5))
gs = fig.add_gridspec(1,3)
fig.add_subplot(gs[0,:2])
plt.semilogx(py.noise.data.f, np.diag(dd_p), '.-')
fig.add_subplot(gs[0,2])
plt.pcolormesh(dd_p)
plt.subplots_adjust(wspace=0.4)

### Levels

In [ ]:
# Create problem
prob_l = om.Problem()
comp = make_component(julia.Levels(py.settings, py.noise.data))
prob_l.model.add_subsystem("l", comp)
prob_l.setup(force_alloc_complex=False)

prob_l.set_val('l.rho_0', rho_0[0])
prob_l.set_val('l.c_0', c_0[0])
prob_l.set_val('l.msap_prop', msap_prop)

prob_l.run_model()
# prob_l.check_partials(compact_print=True, method='fd')

In [ ]:
dd_l = prob_l.compute_totals(of='l.pnlt', wrt='l.msap_prop', return_format='array')


In [ ]:
fig = plt.figure(figsize=(15,5))
plt.semilogx(py.noise.data.f, dd_l.flatten(), '.-')
plt.grid()
plt.plot([py.noise.data.f[10], py.noise.data.f[10]],[-0.17, 0.4], 'k--')
plt.plot([py.noise.data.f[14], py.noise.data.f[14]],[-0.17, 0.4] , 'k--')
plt.ylim([-0.17, 0.4])